# Visualize Notebook
In this notebook the Gold Dataframe will be read and extract information of it. The objective is to see the correlations between the variables and the GDP and also what countries have the highest correlation value.


## Imports
Start importing all the libraries and also the methods of pvalue and search indicators that will be used later in the notebook.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from scipy.stats import shapiro, pearsonr, spearmanr
import os
import statistics
import seaborn as sns
from scipy.stats import norm
import plotly.express as px
from IPython.display import display_html
import warnings
warnings.filterwarnings("ignore")
import ipywidgets as widgets
from ipywidgets import Layout
from ipywidgets import AppLayout, Button, GridspecLayout
from ipywidgets import interact, interact_manual
from Project.Utils.visualize import  search


### Variables that can be changed
The p-value can be changed to a desired value. For the default value is recommnended 0.05. If you desire a higher confidence level lower this variable. This variable reffers directly to significance level, but for porpouses of clarification is set to this name:

In [2]:
PVALUE_VAR = 0.01

## Correlation dataframe.
This dataframe is the main piece of the notebook. Consists in generating for every country the correlation matrix for it and saving only the correlation value of the different variables with the GDP. 
This codeblock also calculates the p-value of Spearman and Pearson, if the value > PVALUE_VAR the correlation will be deleted due to not having statiscal significance.

Later on is concatenated and generates the following result:

In [3]:
#One dataframe per country

write_path = os.getcwd() + '/Output/'

col_country = 'Country'
col_year = 'Year'
col_region = 'Region'
col_gdp = 'GDP'

df= pd.read_csv (write_path + 'GoldDataframe.csv')
corr_df = pd.DataFrame()
corr_df.index.names = [col_country]

#List all the countries, none repeated
countries = set(df[col_country].to_list())

country_dict = {}
corr_dict = {}

for country in countries:
    #Get the DataFrame for a given country
    country_df = df.loc[df[col_country] == country]

    #Correlation matrix for that country
    country_corr_df = country_df.corr()

    #Significance for the correlations
    pval = country_df.corr(method = lambda x, y: pearsonr(x, y)[1]) - np.eye(*country_corr_df.shape)
    p = pval.applymap(lambda x: 1 if x < PVALUE_VAR else np.NaN)
    country_corr_df = country_corr_df * p

    #Trim it into a single row
    country_corr_df = country_corr_df.rename(columns = {col_gdp: country}).drop(index = [col_year, col_gdp])

    #Add the row to a new DataFrame with the correlations for each country
    corr_df = pd.concat([corr_df, country_corr_df[country]], axis = 1)

#Transpose the resulting DataFrame to have the desired format, save it and show it
corr_df = corr_df.transpose()
corr_df.to_csv(os.getcwd()+'/Output/Corr_DF.csv')
corr_df


,AgriShareGDP,CreditToAgriFishForest,EmploymentRural,TotalAgri,% Soldiers,Birth Rate,Death Rate,Homicides,Life Expectancy,Maternal Death Risk,...,% Population Growth,% Rural Population,Civil Liberties,Freedom of Expression,% Healthcare Investment,% Employment Industry,% Education Expenditure,% Men Employment,% Women Employment,Population
Russia,-0.930642,NaN,NaN,0.979737,NaN,0.775658,-0.804394,-0.885042,0.778920,-0.859358,...,0.878687,-0.572207,NaN,-0.697507,NaN,-0.704384,NaN,0.856897,0.877261,NaN
Ireland,-0.721866,0.757281,NaN,0.866601,-0.917973,-0.627465,-0.741050,NaN,0.918794,-0.649948,...,NaN,-0.930707,0.776979,NaN,0.775556,-0.724210,NaN,NaN,0.652968,0.907705
Georgia,-0.950866,0.684907,NaN,NaN,NaN,0.892911,0.906141,NaN,0.889794,NaN,...,0.757804,-0.920270,0.854252,0.702861,NaN,0.902297,NaN,NaN,NaN,-0.955508
Qatar,NaN,NaN,NaN,0.702969,-0.909210,-0.937375,-0.937474,NaN,0.914261,-0.931513,...,NaN,-0.937258,NaN,NaN,NaN,0.793261,NaN,NaN,NaN,0.932368
Kiribati,0.644179,NaN,NaN,NaN,NaN,-0.672807,-0.921030,NaN,0.923800,-0.924940,...,NaN,-0.875490,NaN,NaN,-0.811530,NaN,NaN,NaN,NaN,0.941253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yemen,NaN,NaN,NaN,0.675176,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.784325,0.837351,NaN,NaN,NaN,0.595893
Nicaragua,NaN,-0.827828,NaN,0.976583,-0.920160,-0.960346,-0.720096,-0.700032,0.970542,-0.926781,...,-0.891714,-0.970781,-0.608949,-0.912412,0.931079,-0.849290,NaN,NaN,NaN,0.973662
Lebanon,NaN,NaN,NaN,0.891247,-0.939681,NaN,NaN,NaN,0.888425,NaN,...,NaN,-0.985057,0.890831,0.634427,0.896726,0.899102,NaN,NaN,NaN,0.958025
Morocco,NaN,NaN,NaN,0.864131,-0.984657,-0.778574,-0.977643,NaN,0.988583,-0.976531,...,0.696034,-0.946684,NaN,NaN,0.843802,0.811457,NaN,NaN,NaN,0.937438


In [4]:
#One dataframe per country

write_path = os.getcwd() + '/Output/'

col_country = 'Country'
col_year = 'Year'
col_region = 'Region'
col_gdp = 'GDP'

df= pd.read_csv (write_path + 'GoldDataframe.csv')
corr_df_spear = pd.DataFrame()
corr_df_spear.index.names = [col_country]

#List all the countries, none repeated
countries = set(df[col_country].to_list())

country_dict = {}
corr_dict = {}

for country in countries:
    #Get the DataFrame for a given country
    country_df = df.loc[df[col_country] == country]

    #Correlation matrix for that country
    country_corr_df = country_df.corr(method='spearman')

    #Significance for the correlations
    pval = country_df.corr(method = lambda x, y: spearmanr(x, y)[1]) - np.eye(*country_corr_df.shape)
    p = pval.applymap(lambda x: 1 if x < PVALUE_VAR else np.NaN)
    country_corr_df = country_corr_df * p

    #Trim it into a single row
    country_corr_df = country_corr_df.rename(columns = {col_gdp: country}).drop(index = [col_year, col_gdp])

    #Add the row to a new DataFrame with the correlations for each country
    corr_df_spear = pd.concat([corr_df_spear, country_corr_df[country]], axis = 1)

#Transpose the resulting DataFrame to have the desired format, save it and show it
corr_df_spear = corr_df_spear.transpose()
corr_df_spear.to_csv(os.getcwd()+'/Output/Corr_DF.csv')
corr_df_spear

,AgriShareGDP,CreditToAgriFishForest,EmploymentRural,TotalAgri,% Soldiers,Birth Rate,Death Rate,Homicides,Life Expectancy,Maternal Death Risk,...,% Population Growth,% Rural Population,Civil Liberties,Freedom of Expression,% Healthcare Investment,% Employment Industry,% Education Expenditure,% Men Employment,% Women Employment,Population
Russia,-0.926924,NaN,NaN,0.976623,NaN,0.694635,-0.739441,-0.765930,0.716883,-0.730949,...,0.794805,-0.720779,NaN,-0.702176,NaN,-0.571990,NaN,0.729112,0.763269,NaN
Ireland,-0.659519,NaN,NaN,0.854173,-0.950895,NaN,-0.553907,NaN,0.914555,NaN,...,NaN,-0.915232,0.783265,NaN,0.796880,-0.845882,NaN,NaN,0.715280,0.915232
Georgia,-0.877558,0.706424,NaN,NaN,NaN,0.783371,0.760637,NaN,0.907792,NaN,...,0.678792,-0.903896,0.818781,0.644575,NaN,0.809125,NaN,NaN,NaN,-0.907792
Qatar,-0.594349,NaN,NaN,0.769730,-0.892068,-0.836364,-0.924675,NaN,0.836364,-0.843653,...,NaN,-0.836364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.836364
Kiribati,0.726210,NaN,NaN,NaN,NaN,-0.744398,-0.885714,NaN,0.885714,-0.897073,...,NaN,-0.885714,NaN,NaN,-0.798686,NaN,NaN,NaN,NaN,0.885714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yemen,NaN,NaN,NaN,0.657143,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.757143,0.836364,NaN,NaN,NaN,NaN
Nicaragua,NaN,-0.854355,NaN,0.951948,-0.963590,-0.959740,-0.815200,-0.602627,0.959740,-0.972644,...,-0.920677,-0.959740,-0.835395,-0.940145,0.904839,-0.605215,NaN,NaN,NaN,0.959740
Lebanon,NaN,NaN,NaN,0.927273,-0.986997,NaN,NaN,NaN,0.998701,NaN,...,NaN,-0.998701,0.874272,NaN,0.929780,0.814337,NaN,NaN,NaN,0.994805
Morocco,NaN,NaN,NaN,0.667532,-0.972693,-0.905195,-0.979221,NaN,0.975325,-0.973947,...,0.698701,-0.975325,0.563959,NaN,0.617934,0.876226,NaN,NaN,NaN,0.975325


## Cleaned GoldDataframe 
Before the correlation dataframe was generated and it detected all the correlations with a high p-value. To perform all the following analysis a clean GoldDataframe is needed. It simply detects if the correlation for the indicator and country is Nan and procedes to delete the whole column indicator for the country.

In [5]:

df = df.copy()
for country in countries:
    for ind in corr_df.columns:
        if np.isnan(corr_df[ind][country]):
            df.loc[df[col_country] == country, [ind]] = np.NaN

df.set_index([col_country, col_region, col_year]).to_csv(write_path + 'GoldDataframe_Clean.csv')
df.set_index([col_country, col_region, col_year])



AgriShareGDP  CreditToAgriFishForest  \
Country     Region             Year                                         
Afghanistan South Asia         2000      54.06300                     NaN   
                               2001      54.06300                     NaN   
                               2002      45.13440                     NaN   
                               2003      41.90340                     NaN   
                               2004      35.61280                     NaN   
...                                           ...                     ...   
Zimbabwe    Sub-Saharan Africa 2016       7.87399                     NaN   
                               2017       8.34095                     NaN   
                               2018       8.30469                     NaN   
                               2019       8.17322                     NaN   
                               2020      10.93630                     NaN   

                                     EmploymentRural           GDP  TotalAgri  \
Country     Region             Year                                             
Afghanistan South Asia         2000              NaN   3342.034168        NaN   
                               2001              NaN   3598.470576        NaN   
                               2002              NaN   4141.523943        NaN   
                               2003              NaN   4729.042179        NaN   
                               2004              NaN   5388.482107        NaN   
...                                              ...           ...        ...   
Zimbabwe    Sub-Saharan Africa 2016              NaN  20548.678073        NaN   
                               2017              NaN  22040.902301        NaN   
                               2018              NaN  24311.560545        NaN   
                               2019              NaN  21935.075306        NaN   
                               2020              NaN  21786.743637        NaN   

                                     % Soldiers  Birth Rate  Death Rate  \
Country     Region             Year                                       
Afghanistan South Asia         2000         NaN      48.021      11.718   
                               2001         NaN      47.505      11.387   
                               2002         NaN      46.901      11.048   
                               2003         NaN      46.231      10.704   
                               2004         NaN      45.507      10.356   
...                                         ...         ...         ...   
Zimbabwe    Sub-Saharan Africa 2016     0.75975         NaN       8.286   
                               2017     0.75072         NaN       8.044   
                               2018     0.73821         NaN       7.883   
                               2019     0.73821         NaN       7.773   
                               2020     0.73821         NaN       7.692   

                                     Homicides  Life Expectancy  ...  \
Country     Region             Year                              ...   
Afghanistan South Asia         2000        NaN           55.841  ...   
                               2001        NaN           56.308  ...   
                               2002        NaN           56.784  ...   
                               2003        NaN           57.271  ...   
                               2004        NaN           57.772  ...   
...                                        ...              ...  ...   
Zimbabwe    Sub-Saharan Africa 2016        NaN           60.294  ...   
                               2017        NaN           60.812  ...   
                               2018        NaN           61.195  ...   
                               2019        NaN           61.490  ...   
                               2020        NaN           61.738  ...   

                                     % Population Growth  % Rural Po

# Analysis
From now on all the data will be analysed with the Cleaned dataframes.

## Choropleth map
This block uses the IppyWidget for the indicator list and the Choroplet for the Map. 

In this section a map is generated and painted with the values of the indicator. You can salect the indicator from the list and an new map is generated, also along 2 lists of the highest and lowest countries for the selected indicator. 




In [6]:
indicator = widgets.SelectMultiple(
    options = corr_df.columns.tolist(),
    value = ['AgriShareGDP'],
    description='Indicator',
    disabled=False,
    layout = Layout(width='50%', height='80px')
)



def globalGrapgh(indicator):
    ind = indicator[0]
    N = 10
    fig = px.choropleth(corr_df, locations = corr_df.index, locationmode='country names', 
                        color= indicator[0],projection="natural earth",
                        color_continuous_scale='RdBu',
                        width=700, height=500)

    pos_corr = corr_df.drop(corr_df.columns.difference([ind]), axis = 1).sort_values(by = ind, axis = 0, ascending = False).head(n = N)
    neg_corr = corr_df.drop(corr_df.columns.difference([ind]), axis = 1).sort_values(by = ind, axis = 0, ascending = True).head(n = N)

    fig.update(layout_coloraxis_showscale=True)
    fig.show()
    

    pos_styler = pos_corr.style.set_table_attributes("style='display:inline'").set_caption('Direct correlation')
    neg_styler = neg_corr.style.set_table_attributes("style='display:inline'").set_caption('Inverse correlation')

    space = "\xa0" * 10
    display_html(pos_styler._repr_html_()+ space  + neg_styler._repr_html_(), raw=True)


widgets.interactive(globalGrapgh, indicator = indicator)

interactive(children=(SelectMultiple(description='Indicator', index=(0,), layout=Layout(height='80px', width='…

### Choropleth for population

This map represents the population of the world. It is usefull for analysing the data if the aggregation is calculated later by the population, because the countries with a higher population have more impact in the global indicators.

In [7]:
df_gold = pd.read_csv(write_path + '/GoldDataframe.csv')
fig = px.choropleth(df_gold, locations="Country", locationmode='country names', 
                     color="Population", hover_name="Country",projection="natural earth",
                     animation_frame="Year",width=800, height=500,
                     color_continuous_scale='Reds',
                     range_color=[1000,340000000])
fig.update(layout_coloraxis_showscale=True)
fig.show()

## Country Indicators
In this codeblock a widged is implmented to generate a table with the indicators. In order to make it work just select the country from the dropdown and procede to establasih the threshold. The default value for the threshold is 0.7, which we consider to be the minimum to consider an indicator correlated to the GDP. To analyze the results:
- H0: the indicator and the GDP are uncorrelated.​
- H1: the indicator and the GDP are correlated.​

P-value: is the probability of obtaining  test results at least as extreme as the result actually observed.​
Confidence level: probability that a population parameter will fall between a set of values for a certain proportion of times. ​
Significance level:  probability of the study rejecting the null hypothesis when it is actually true.​


Confidence level is set to 1 - PVALUE_VAR% .  Significance level α = PVALUE_VAR

If p-value < α then reject  H0 and accept H1.​

The code consists in a IppyWidget with a slider and a dropdown. Once this parameters have been set it calls the method 'search' and applies a style format of the returned Dataframe.

In [8]:
def tableOut(Threshold, Country):

    df = search(Threshold, 'Country', Country)
    if df.empty:
        return print("No indicators have been found.")

    left = pd.Series([PVALUE_VAR, PVALUE_VAR], index=['P-value Pearson', 'P-value Spearman'])
    left2 = pd.Series([-1, -1], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    left3 = pd.Series([0, 0], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    df =df.style.highlight_between(left=left, right=1.5, axis=1, props='color:white; background-color:red;')\
        .highlight_between(left=left2, right=1.5, axis=1, props='color:white; background-color:#929bfc;')\
        .highlight_between(left=left3, right=1.5, axis=1, props='color:white; background-color:#b3b9ff;')\
        .format('{:,.4f}', subset = ['GDP Pearson Corr', 'GDP Spearman Corr'])\
        .format('{:,.12f}', subset = ['P-value Pearson', 'P-value Spearman']) 
    
    display(df)

    

@interact(
    Country = sorted(corr_df.index.tolist()),
    Threshold = (0, 1, 0.05))
def g(Country = 'Afghanistan', Threshold = 0.7):
    return tableOut(Threshold,Country)

    
        

interactive(children=(Dropdown(description='Country', options=('Afghanistan', 'Albania', 'Algeria', 'Andorra',…

## Region Indicators
It's almost the same codeblock as before, but for the regions. The groups have been formed according to the Dataframe of World Data Bank.


In [9]:
def tableOut2(Threshold, Region):

    df = search(Threshold, 'Region', Region)
    if df.empty:
        return print("No indicators have been found.")

    left = pd.Series([PVALUE_VAR, PVALUE_VAR], index=['P-value Pearson', 'P-value Spearman'])
    left2 = pd.Series([-1, -1], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    left3 = pd.Series([0, 0], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    df = df.style.highlight_between(left=left, right=1.5, axis=1, props='color:white; background-color:red;')\
        .highlight_between(left=left2, right=1.5, axis=1, props='color:white; background-color:#929bfc;')\
        .highlight_between(left=left3, right=1.5, axis=1, props='color:white; background-color:#b3b9ff;')\
        .format('{:,.4f}', subset = ['GDP Pearson Corr', 'GDP Spearman Corr'])\
        .format('{:,.12f}', subset = ['P-value Pearson', 'P-value Spearman']) 
    
    display(df)



@interact(
    Region = set(df['Region'].to_list()),
    Threshold = (0, 1, 0.05))
def g(Region = 'East Asia and Pacific', Threshold = 0.7):
    return tableOut2(Threshold, Region)

    

interactive(children=(Dropdown(description='Region', index=3, options=('Europe and Central Asia', 'Middle East…

## Global Indicators
This section reperesents the final analysis of the indicators. It shows in the indicators table with the highest correlations. With this results we can give an answer to the hypothesis of the project and establish the indicators with a high GDP relation.

In [10]:
def tableOut3(Threshold):

    df = search(Threshold, 'Global')
    if df.empty:
        return print("No indicators have been found.")

    left = pd.Series([PVALUE_VAR, PVALUE_VAR], index=['P-value Pearson', 'P-value Spearman'])
    left2 = pd.Series([-1, -1], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    left3 = pd.Series([0, 0], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    df =df.style.highlight_between(left=left, right=1.5, axis=1, props='color:white; background-color:red;')\
        .highlight_between(left=left2, right=1.5, axis=1, props='color:white; background-color:#929bfc;')\
        .highlight_between(left=left3, right=1.5, axis=1, props='color:white; background-color:#b3b9ff;')\
        .format('{:,.4f}', subset = ['GDP Pearson Corr', 'GDP Spearman Corr'])\
        .format('{:,.12f}', subset = ['P-value Pearson', 'P-value Spearman']) 
    
    display(df)



@interact(
    Threshold = (0, 1, 0.05))
def g(Threshold = 0.7):
    return tableOut3(Threshold)


interactive(children=(FloatSlider(value=0.7, description='Threshold', max=1.0, step=0.05), Output()), _dom_cla…

## Median Global Indicators
Reading the corr_df generated at the beggining of this notebook it calculates the median of each column. This approximation is very simple and might give an idea if the aggregation method is correct. Obviously the result won't be the same but the tendency has to be similar.

Having this in mind for each column in the corr_df import the name of the indicator and compute the median. Save this new dataframe in result_df.

In [11]:
result_df = pd.DataFrame()
for column in corr_df.columns:
    aux = pd.DataFrame({'Indicator': [column],
                        'GDP Pearson Corr': [corr_df[column].median()],
                        'GDP Spearman Corr': [corr_df_spear[column].median()]})
    result_df = pd.concat([result_df, aux], ignore_index=False, axis = 0)
    result_df = result_df.sort_values(by=["GDP Pearson Corr"], ascending = False)
    

result_df.set_index(['Indicator'], inplace=True)

Using a IppyWidget display the previously generated result_df. Lowering the Threshold more than 0.7 will show indicators that don't have a real correlation with the GDP, procede with caution extracting conclusions of them.

In [12]:
def tableOutMedian(Threshold):
    df = pd.concat([result_df.loc[(result_df['GDP Pearson Corr'] >= Threshold) & (result_df['GDP Spearman Corr'] >= Threshold)], result_df.loc[(result_df['GDP Pearson Corr'] <= -Threshold) & (result_df['GDP Pearson Corr'] <= -Threshold)]], axis = 0)

    if df.empty:
        return print("No indicators have been found.")

    left2 = pd.Series([-1, -1], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    left3 = pd.Series([0, 0], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    df =df.style.highlight_between(left=left2, right=1.5, axis=1, props='color:white; background-color:#929bfc;')\
        .highlight_between(left=left3, right=1.5, axis=1, props='color:white; background-color:#b3b9ff;')\
        .format('{:,.4f}', subset = ['GDP Pearson Corr', 'GDP Spearman Corr'])\

    display(df)



@interact(
    Threshold = (0, 1, 0.05))
def g(Threshold = 0.7):
    return tableOutMedian(Threshold)

interactive(children=(FloatSlider(value=0.7, description='Threshold', max=1.0, step=0.05), Output()), _dom_cla…